# Init stuffs

### Imports

### Input data

In [1]:
filename = "./input_1.txt"
test_filename = "./input_2.txt"

with open(file=filename) as f:
    data = f.read().splitlines()

with open(file=test_filename) as f:
    test_data = f.read().splitlines()

# Pt. 1

In [2]:
DELTAS = {"^": [0, -1], ">": [1,0], "v": [0,1], "<": [-1,0]}

def walk_longest_route(grid: list[list[str]], follow_slopes: bool = True) -> tuple[int, list[tuple]]:
    active_paths = [[(1,0)]]
    found_paths = []

    while active_paths:
        current_path = active_paths.pop()
        previous_grid_char = grid[current_path[-1][1]][current_path[-1][0]]
        for delta in DELTAS.values():
            new_x, new_y = [current_path[-1][i]+delta[i] for i in range(2)]
            if (new_x, new_y) in current_path:
                continue

            if not 0 <= new_x < len(grid[0]) or not 0 <= new_y < len(grid):
                continue

            if previous_grid_char in DELTAS.keys() and follow_slopes:
                if delta != DELTAS[previous_grid_char]:
                    continue

            next_grid_char = grid[new_y][new_x]
            if next_grid_char == "#":
                continue

            if new_y == len(grid) - 1:
                found_paths.append(current_path)
                continue

            active_paths.append(current_path+[(new_x, new_y)])
    
    return max([len(path) for path in found_paths]), found_paths


In [3]:
grid = data
max_length, paths = walk_longest_route(grid=grid)
max_length

2354

# Init pt. 2

### More imports?

### Input data (again)

In [4]:
# filename = "./input_2.txt"

# with open(file=filename) as f:
#     input_data_pt2 = f.read().splitlines()

# Pt. 2

In [29]:
def find_nodes(grid: list[list[str]]) -> dict:
    found_nodes = 2
    nodes = {
        0: {
            "location": (*[i for i in range(len(data[0])) if data[0][i] == "."], 0),
            "connects_to": {}
            },
        1: {
            "location": (*[i for i in range(len(grid[0])) if data[len(grid)-1][i] == "."], len(grid)-1),
            "connects_to": {}
        }
    }
    for y in range(1,len(grid)-1):
        for x in range(1,len(grid[0])-1):
            if grid[y][x] != "#":
                neighbors = 0
                for delta in DELTAS.values():
                    if grid[y+delta[1]][x+delta[0]] != "#":
                        neighbors += 1
                if neighbors > 2:
                    nodes[found_nodes] = {"location": (x,y), "connects_to": {}}
                    found_nodes += 1
    return nodes

def follow_path(grid: list[list[str]], nodes: dict, path: list[tuple]) -> tuple[int,int]:
    while path[-1] not in [v["location"] for v in nodes.values()]:
        for delta in DELTAS.values():
            new_x, new_y = [path[-1][i]+delta[i] for i in range(2)]
            if (new_x, new_y) in path:
                continue

            if not 0 <= new_x < len(grid[0]) or not 0 <= new_y < len(grid):
                continue

            next_grid_char = grid[new_y][new_x]
            if next_grid_char == "#":
                continue

            path.append((new_x, new_y))
            break

    return (*[k for k,v in nodes.items() if v["location"] == path[-1]], len(path)-1, (path[-1][0]-path[-2][0], path[-1][1]-path[-2][1]))
    

def connect_nodes(grid: list[list[str]], nodes: dict) -> dict:
    local_nodes = nodes.copy()
    for k, v in local_nodes.items():
        for delta in DELTAS.values():
            if delta in [connection[1] for connection in v["connects_to"].values()]:
                continue

            new_x = v["location"][0]+delta[0]
            new_y = v["location"][1]+delta[1]

            if not 0 <= new_x < len(grid[0]) or not 0 <= new_y < len(grid):
                continue

            if grid[new_y][new_x] != "#":
                path = [v["location"], (new_x, new_y)]
                next_node, distance, last_direction = follow_path(grid=grid, nodes=local_nodes, path=path)
                local_nodes[k]["connects_to"][next_node] = (distance, (delta[0], delta[1]))
                local_nodes[next_node]["connects_to"][k] = (distance, (-last_direction[0], -last_direction[1]))
    return local_nodes

def find_longest_path(nodes: dict) -> int:
    routes = [[[0], 0]]
    found_routes = []
    while routes:
        route = routes.pop()
        if route[0][-1] == 1:
            found_routes.append(route)
            continue
        
        connectable_nodes = [k for k in nodes[route[0][-1]]["connects_to"].keys() if k not in route[0]]

        if len(connectable_nodes) == 0:
            continue

        for next_node in connectable_nodes:
            routes.append([route[0]+[next_node], route[1]+nodes[route[0][-1]]["connects_to"][next_node][0]])
    return found_routes

In [22]:
nodes = find_nodes(grid=grid)
connected_nodes = connect_nodes(grid=grid, nodes=nodes)
long_paths = find_longest_path(nodes=connected_nodes)
max([path[-1] for path in long_paths])
